<a href="https://colab.research.google.com/github/kopalgarg/GAN-keras/blob/main/GAN_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## T81-558: Applications of Deep Neural Networks

Module 7: Generative Adversarial Networks

Part 7.1: Introduction to GANS for Image and Data Generation



In [5]:
# mount Google drive for use with Colab
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google Colab")
  %tensorflow_version 2.x
except:
  print("Note: not using Google Colab")
  COLAB = False

Mounted at /content/drive
Note: using Google Colab


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation